# 5.10 프로젝트 : 뉴스기사 분야별로 크롤링하여 저장!

In [1]:
from urllib import parse

header = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36'}
# Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.131 Safari/537.36
def getDownload(url, params={}, retries=3):
    resp = None
    
    try:
        resp = requests.get(url, params=params, headers=header)
        resp.raise_for_status()
    except requests.exceptions.HTTPError as e:
        if 500 <= e.response.status_code < 600 and retries > 0:
            print(retries)
            resp = getDownload(url, params, retries-1)
        else:
            print(e.response.status_code)
            print(e.response.reason)
            print(e.response.headers)
            
    return resp

# 크롤 관련 라이브러리들 import

In [2]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time

# 다음
## 사회, 정치, 경제, 국제, 문화, IT로 나눠서

In [ ]:
def daum_news_detail_crawl(links):
    for link in links:
        # 뉴스로만 향하는 링크들만
        if link.startswith("http://v.media.daum.net/v/"):

            article_id = link.split("/")[-1]

            html = getDownload(link)
            detail_dom = BeautifulSoup(html.content, "lxml")

            # 파일 생성
            file_name = "./news_crawl_project/" + field + "-" + article_id
            f = open(file_name, 'w', encoding="utf-8")

            for _ in detail_dom.select("#harmonyContainer > section > p"):
    #           각 p태그 파일에 write
                f.write(_.text)

            print(file_name, "생성 완료")
            time.sleep(1)
            f.close()

In [ ]:
browser = webdriver.Chrome()
url = "https://media.daum.net/"
browser.get(url)

### 사회

In [ ]:
# 분야이름 
field = browser.find_elements_by_css_selector("#kakaoGnb > div > ul > li:nth-child(2) > a")[0].text
# 사회분야 클릭
browser.find_elements_by_css_selector("#kakaoGnb > div > ul > li:nth-child(2) > a")[0].click()

### 페이지 소스 받아오기

In [ ]:
dom = BeautifulSoup(browser.page_source, "lxml")

### 세부링크 가져와서 urljoin -> url방문 -> 내용 긁어오기
#### 처음엔 생으로 짜고 이것을 함수로 만들기

In [ ]:
links = [_["href"] for _ in dom.select(".section_cate.section_headline .link_txt") if _.has_attr('href')]

for link in links:
    # 뉴스로만 향하는 링크들만
    if link.startswith("http://v.media.daum.net/v/"):

        article_id = link.split("/")[-1]

        html = getDownload(link)
        detail_dom = BeautifulSoup(html.content, "lxml")
        
        # 파일 생성
        file_name = "./news_crawl_project/" + field + "-" + article_id
        f = open(file_name, 'w', encoding="utf-8")
        
        for _ in detail_dom.select("#harmonyContainer > section > p"):
#           각 p태그 파일에 write
            f.write(_.text)
        
        print(file_name, "생성 완료")
        
        f.close()
        #harmonyContainer > section > p:nth-child(3)
        

## 정치

In [ ]:
field = browser.find_elements_by_xpath("//*[@id='kakaoGnb']/div/ul/li[3]/a")[0].text
browser.find_elements_by_xpath("//*[@id='kakaoGnb']/div/ul/li[3]/a")[0].click()

In [ ]:
dom = BeautifulSoup(browser.page_source, "lxml")
links = [_["href"] for _ in dom.select(".section_cate.section_headline .link_txt") if _.has_attr('href')]

daum_news_detail_crawl(links)

## 경제

In [ ]:
field = browser.find_elements_by_css_selector("#kakaoGnb > div > ul > li:nth-child(4) > a")[0].text
browser.find_elements_by_css_selector("#kakaoGnb > div > ul > li:nth-child(4) > a")[0].click()

dom = BeautifulSoup(browser.page_source, "lxml")

links = [_["href"] for _ in dom.select(".section_cate.section_headline .link_txt") if _.has_attr('href')]

daum_news_detail_crawl(links)

## 국제

In [ ]:
field = browser.find_elements_by_css_selector("#kakaoGnb > div > ul > li:nth-child(5) > a")[0].text
browser.find_elements_by_css_selector("#kakaoGnb > div > ul > li:nth-child(5) > a")[0].click()

dom = BeautifulSoup(browser.page_source, "lxml")

links = [_["href"] for _ in dom.select(".section_cate.section_headline .link_txt") if _.has_attr('href')]

daum_news_detail_crawl(links)

## 문화

In [ ]:
field = browser.find_elements_by_css_selector("#kakaoGnb > div > ul > li:nth-child(6) > a")[0].text
browser.find_elements_by_css_selector("#kakaoGnb > div > ul > li:nth-child(6) > a")[0].click()

dom = BeautifulSoup(browser.page_source, "lxml")

links = [_["href"] for _ in dom.select(".section_cate.section_headline .link_txt") if _.has_attr('href')]

daum_news_detail_crawl(links)

        #harmonyContainer > section > p:nth-child(3)
        

## IT

In [ ]:
field = browser.find_elements_by_css_selector("#kakaoGnb > div > ul > li:nth-child(7) > a")[0].text
browser.find_elements_by_css_selector("#kakaoGnb > div > ul > li:nth-child(7) > a")[0].click()

dom = BeautifulSoup(browser.page_source, "lxml")

daum_news_detail_crawl(links)

# 네이트 뉴스

In [3]:
browser = webdriver.Chrome()
url = "https://news.nate.com"
browser.get(url)

In [4]:
def nate_news_detail_crawl(links):
    print(field, "분야 개수 : ", len(links))
    for link in links:
        if link.startswith("//news.nate.com/view/"):
            article_id = link.split("/")[-1].split('?')[0]

            html = getDownload("https:" + link)
            detail_dom = BeautifulSoup(html.content, "lxml")

            # 파일 생성
            file_name = "./news_crawl_project/" + field + "-" + article_id
            f = open(file_name, 'w', encoding="utf-8")

            for _ in detail_dom.select("#realArtcContents"):
    #           각 p태그 파일에 write
                f.write(_.text)

            print(file_name, "생성 완료")
            time.sleep(1)

### 사회

In [5]:
field = browser.find_element_by_class_name("news03").text
browser.find_element_by_class_name("news03").click()

In [6]:
dom = BeautifulSoup(browser.page_source, "lxml")

links = [_["href"] for _ in dom.select("#newsContents a") if _.has_attr('href')]

nate_news_detail_crawl(links)

정치 분야 개수 :  43
./news_crawl_project/정치-20190510n33943 생성 완료
./news_crawl_project/정치-20190510n34694 생성 완료
./news_crawl_project/정치-20190510n33811 생성 완료
./news_crawl_project/정치-20190510n30801 생성 완료
./news_crawl_project/정치-20190510n33982 생성 완료
./news_crawl_project/정치-20190510n28476 생성 완료
./news_crawl_project/정치-20190510n22582 생성 완료
./news_crawl_project/정치-20190510n10775 생성 완료
./news_crawl_project/정치-20190510n33288 생성 완료
./news_crawl_project/정치-20190510n19472 생성 완료
./news_crawl_project/정치-20190510n17916 생성 완료
./news_crawl_project/정치-20190510n11053 생성 완료
./news_crawl_project/정치-20190510n11654 생성 완료
./news_crawl_project/정치-20190508n28159 생성 완료
./news_crawl_project/정치-20190508n30739 생성 완료
./news_crawl_project/정치-20190508n21728 생성 완료
./news_crawl_project/정치-20190510n30241 생성 완료
./news_crawl_project/정치-20190510n31292 생성 완료
./news_crawl_project/정치-20190510n28476 생성 완료
./news_crawl_project/정치-20190510n17741 생성 완료
./news_crawl_project/정치-20190510n21961 생성 완료
./news_crawl_project/정치-20190510n05792 생

### 경제

In [7]:
field = browser.find_element_by_class_name("news04").text
browser.find_element_by_class_name("news04").click()

dom = BeautifulSoup(browser.page_source, "lxml")

links = [_["href"] for _ in dom.select("#newsContents a") if _.has_attr('href')]

nate_news_detail_crawl(links)

경제 분야 개수 :  31
./news_crawl_project/경제-20190510n23962 생성 완료
./news_crawl_project/경제-20190510n16085 생성 완료
./news_crawl_project/경제-20190509n30489 생성 완료
./news_crawl_project/경제-20190509n26166 생성 완료
./news_crawl_project/경제-20190508n17337 생성 완료
./news_crawl_project/경제-20190505n02259 생성 완료
./news_crawl_project/경제-20190510n20716 생성 완료
./news_crawl_project/경제-20190509n21628 생성 완료
./news_crawl_project/경제-20190509n08358 생성 완료
./news_crawl_project/경제-20190508n10724 생성 완료
./news_crawl_project/경제-20190429n36193 생성 완료
./news_crawl_project/경제-20190424n03150 생성 완료
./news_crawl_project/경제-20190510n22323 생성 완료
./news_crawl_project/경제-20190509n43393 생성 완료
./news_crawl_project/경제-20190509n37082 생성 완료
./news_crawl_project/경제-20190509n20700 생성 완료
./news_crawl_project/경제-20190509n30055 생성 완료
./news_crawl_project/경제-20190509n13451 생성 완료
./news_crawl_project/경제-20190510n21561 생성 완료
./news_crawl_project/경제-20190509n16568 생성 완료
./news_crawl_project/경제-20190509n18588 생성 완료
./news_crawl_project/경제-20190509n00892 생

### 사회

In [8]:
field = browser.find_element_by_class_name("news05").text
browser.find_element_by_class_name("news05").click()

dom = BeautifulSoup(browser.page_source, "lxml")

links = [_["href"] for _ in dom.select("#newsContents a") if _.has_attr('href')]

nate_news_detail_crawl(links)

사회 분야 개수 :  35
./news_crawl_project/사회-20190510n29526 생성 완료
./news_crawl_project/사회-20190510n19983 생성 완료
./news_crawl_project/사회-20190510n20307 생성 완료
./news_crawl_project/사회-20190510n09675 생성 완료
./news_crawl_project/사회-20190510n02247 생성 완료
./news_crawl_project/사회-20190509n42190 생성 완료
./news_crawl_project/사회-20190509n42453 생성 완료
./news_crawl_project/사회-20190510n28426 생성 완료
./news_crawl_project/사회-20190510n03290 생성 완료
./news_crawl_project/사회-20190510n00686 생성 완료
./news_crawl_project/사회-20190510n00835 생성 완료
./news_crawl_project/사회-20190509n45302 생성 완료
./news_crawl_project/사회-20190509n42242 생성 완료
./news_crawl_project/사회-20190509n42189 생성 완료
./news_crawl_project/사회-20190509n38732 생성 완료
./news_crawl_project/사회-20190510n24778 생성 완료
./news_crawl_project/사회-20190428n07619 생성 완료
./news_crawl_project/사회-20190425n26851 생성 완료
./news_crawl_project/사회-20190425n02644 생성 완료
./news_crawl_project/사회-20190425n05017 생성 완료
./news_crawl_project/사회-20190423n10526 생성 완료
./news_crawl_project/사회-20190422n38414 생

### 세계

In [9]:
field = browser.find_element_by_class_name("news06").text
browser.find_element_by_class_name("news06").click()

dom = BeautifulSoup(browser.page_source, "lxml")

links = [_["href"] for _ in dom.select("#newsContents a") if _.has_attr('href')]

nate_news_detail_crawl(links)

세계 분야 개수 :  40
./news_crawl_project/세계-20190510n36462 생성 완료
./news_crawl_project/세계-20190510n35841 생성 완료
./news_crawl_project/세계-20190510n34762 생성 완료
./news_crawl_project/세계-20190510n30877 생성 완료
./news_crawl_project/세계-20190510n33174 생성 완료
./news_crawl_project/세계-20190509n32096 생성 완료
./news_crawl_project/세계-20190509n38565 생성 완료
./news_crawl_project/세계-20190509n04808 생성 완료
./news_crawl_project/세계-20190509n00287 생성 완료
./news_crawl_project/세계-20190509n02000 생성 완료
./news_crawl_project/세계-20190510n30497 생성 완료
./news_crawl_project/세계-20190510n19645 생성 완료
./news_crawl_project/세계-20190510n13688 생성 완료
./news_crawl_project/세계-20190509n29777 생성 완료
./news_crawl_project/세계-20190509n03529 생성 완료
./news_crawl_project/세계-20190507n20728 생성 완료
./news_crawl_project/세계-20190507n22333 생성 완료
./news_crawl_project/세계-20190507n05523 생성 완료
./news_crawl_project/세계-20190510n24922 생성 완료
./news_crawl_project/세계-20190510n25158 생성 완료
./news_crawl_project/세계-20190510n18926 생성 완료
./news_crawl_project/세계-20190510n22599 생

### IT/과학

In [11]:
field = browser.find_element_by_class_name("news07").text[:2]
browser.find_element_by_class_name("news07").click()

dom = BeautifulSoup(browser.page_source, "lxml")

links = [_["href"] for _ in dom.select("#newsContents a") if _.has_attr('href')]

nate_news_detail_crawl(links)

IT 분야 개수 :  36
./news_crawl_project/IT-20190510n10194 생성 완료
./news_crawl_project/IT-20190510n10283 생성 완료
./news_crawl_project/IT-20190510n00040 생성 완료
./news_crawl_project/IT-20190510n01503 생성 완료
./news_crawl_project/IT-20190509n13265 생성 완료
./news_crawl_project/IT-20190509n05130 생성 완료
./news_crawl_project/IT-20190510n03515 생성 완료
./news_crawl_project/IT-20190509n18569 생성 완료
./news_crawl_project/IT-20190509n02250 생성 완료
./news_crawl_project/IT-20190509n03000 생성 완료
./news_crawl_project/IT-20190510n03568 생성 완료
./news_crawl_project/IT-20190509n03472 생성 완료
./news_crawl_project/IT-20190508n03456 생성 완료
./news_crawl_project/IT-20190503n03443 생성 완료
./news_crawl_project/IT-20190502n04283 생성 완료
./news_crawl_project/IT-20190510n01856 생성 완료
./news_crawl_project/IT-20190510n02270 생성 완료
./news_crawl_project/IT-20190505n02690 생성 완료
./news_crawl_project/IT-20190504n02086 생성 완료
./news_crawl_project/IT-20190502n17987 생성 완료
./news_crawl_project/IT-20190428n01358 생성 완료
./news_crawl_project/IT-20190427n02168 생

# 번외 : 네이버 뉴스
## 이번엔 beautifulsoup만 써서 많이 본 뉴스 가져오기

In [63]:
url = "https://news.naver.com/"
html = getDownload(url)
dom = BeautifulSoup(html.content, "lxml")

### 필드 리스트 따로뽑고 10개씩 짤라서 하기

In [71]:
field_list = dom.select(".category_ranking")[0].text.split(" ")
field_list = list(filter(None, field_list))
len(field_list)

6

### 랭크가 10개씩 있으니 10개될때마다 1로 리셋하면서 긁어오자

In [78]:
dom.select(".section_list_ranking a")[-1]

<a class="nclicks(rig.ranksci)" href="/main/ranking/read.nhn?mid=etc&amp;sid1=111&amp;rankingType=popular_day&amp;oid=366&amp;aid=0000433800&amp;date=20190510&amp;type=1&amp;rankingSeq=10&amp;rankingSectionId=105" title="넷마블, 'BTS월드' 출시 임박…10일부터 국내외 동시 사전등록">넷마블, 'BTS월드' 출시 임박…10일부터 국내외 동시 사전등록</a>

In [77]:
rank = 1
index = 0
field = field_list[index]
links = dom.select(".section_list_ranking a")

for _ in links:
    
    link = requests.compat.urljoin(url, _["href"])
    html = getDownload(link)
    detail_dom = BeautifulSoup(html.content, "lxml")
    
    # /로 필드이름이 있으면 디렉토리로 인식해서 예외문을 만들었다.
    try:
        file_name = "./news_crawl_project/" + field + "-naver" + str(rank)
        f = open(file_name, 'w', encoding="utf-8")
    except FileNotFoundError:
        file_name = "./news_crawl_project/" + field.split("/")[0] + "-naver" + str(rank)
        f = open(file_name, 'w', encoding="utf-8")


    for _ in detail_dom.select("#articleBodyContents"):
        f.write(_.text)

    print(file_name, "생성 완료")
#     time.sleep(1)
   
    rank += 1
    
    # rank랑 field 리셋
    if rank > 10 :
        index += 1
        rank = 1
        if index < len(field_list):
            field = field_list[index]
        
        


./news_crawl_project/정치-naver1 생성 완료
./news_crawl_project/정치-naver2 생성 완료
./news_crawl_project/정치-naver3 생성 완료
./news_crawl_project/정치-naver4 생성 완료
./news_crawl_project/정치-naver5 생성 완료
./news_crawl_project/정치-naver6 생성 완료
./news_crawl_project/정치-naver7 생성 완료
./news_crawl_project/정치-naver8 생성 완료
./news_crawl_project/정치-naver9 생성 완료
./news_crawl_project/정치-naver10 생성 완료
./news_crawl_project/경제-naver1 생성 완료
./news_crawl_project/경제-naver2 생성 완료
./news_crawl_project/경제-naver3 생성 완료
./news_crawl_project/경제-naver4 생성 완료
./news_crawl_project/경제-naver5 생성 완료
./news_crawl_project/경제-naver6 생성 완료
./news_crawl_project/경제-naver7 생성 완료
./news_crawl_project/경제-naver8 생성 완료
./news_crawl_project/경제-naver9 생성 완료
./news_crawl_project/경제-naver10 생성 완료
./news_crawl_project/사회-naver1 생성 완료
./news_crawl_project/사회-naver2 생성 완료
./news_crawl_project/사회-naver3 생성 완료
./news_crawl_project/사회-naver4 생성 완료
./news_crawl_project/사회-naver5 생성 완료
./news_crawl_project/사회-naver6 생성 완료
./news_crawl_project/사회-naver7 생성 완료